In [31]:
#test
import numpy as np
import xarray as xr
import xoak
import geopandas as gpd
#xr.set_options(display_style='text');

In [43]:
shape = (100, 100)
lat = np.random.uniform(-90, 90, size=shape)
lon = np.random.uniform(-180, 180, size=shape)

field = lat + lon

In [44]:
#field.shape
type(field)

numpy.ndarray

In [45]:
ds_mesh = xr.Dataset(
    coords={'lat': (('x', 'y'), lat), 'lon': (('x', 'y'), lon)},
    data_vars={'field': (('x', 'y'), field)},
)

ds_mesh

<xarray.Dataset>
Dimensions:  (x: 100, y: 100)
Coordinates:
    lat      (x, y) float64 19.31 27.73 -10.93 1.032 -18.08 -76.77 86.78 ...
    lon      (x, y) float64 172.2 -91.2 56.69 -127.9 -45.15 -44.48 -5.636 ...
Dimensions without coordinates: x, y
Data variables:
    field    (x, y) float64 191.5 -63.46 45.76 -126.8 -63.22 -121.2 81.14 ...

In [20]:
ds_mesh.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

In [21]:
ds_trajectory = xr.Dataset({
    'latitude': ('trajectory', np.linspace(-10, 40, 30)),
    'longitude': ('trajectory', np.linspace(-150, 150, 30))
})

ds_trajectory

<xarray.Dataset>
Dimensions:    (trajectory: 30)
Dimensions without coordinates: trajectory
Data variables:
    latitude   (trajectory) float64 -10.0 -8.276 -6.552 -4.828 -3.103 -1.379 ...
    longitude  (trajectory) float64 -150.0 -139.7 -129.3 -119.0 -108.6 ...

In [22]:
ds_selection = ds_mesh.xoak.sel(
    lat=ds_trajectory.latitude,
    lon=ds_trajectory.longitude
)

ds_selection

<xarray.Dataset>
Dimensions:  (trajectory: 30)
Coordinates:
    lat      (trajectory) float64 -11.01 -7.519 -7.127 -5.627 -4.798 -2.334 ...
    lon      (trajectory) float64 -150.9 -138.3 -130.0 -120.0 -107.6 -97.93 ...
Dimensions without coordinates: trajectory
Data variables:
    field    (trajectory) float64 -162.0 -145.9 -137.1 -125.7 -112.4 -100.3 ...

In [23]:
ds_trajectory.plot.scatter(x='longitude', y='latitude', c='k', alpha=0.7);
ds_selection.plot.scatter(x='lon', y='lat', hue='field', alpha=0.9);

AttributeError: 'Dataset' object has no attribute 'plot'

In [24]:
type(ds_selection)

xarray.core.dataset.Dataset

In [25]:
ds_selection

<xarray.Dataset>
Dimensions:  (trajectory: 30)
Coordinates:
    lat      (trajectory) float64 -11.01 -7.519 -7.127 -5.627 -4.798 -2.334 ...
    lon      (trajectory) float64 -150.9 -138.3 -130.0 -120.0 -107.6 -97.93 ...
Dimensions without coordinates: trajectory
Data variables:
    field    (trajectory) float64 -162.0 -145.9 -137.1 -125.7 -112.4 -100.3 ...

In [36]:
#extract UA data at points
path_ua = '/Users/carina/Desktop/data/seup/UA_SWE/'
files_ua = path_ua + '4km_SWE_Depth*.nc'
ds_ua = xr.open_mfdataset(files_ua)

In [32]:
df_grid = gpd.read_file('/Users/carina/Desktop/code/SEUP/grid_cells.geojson')
df_grid['geometry']
df_grid.plot()
df_grid

,cell_id,region,geometry
0,0003f387-71c4-48f6-b2b0-d853bd4f0aba,sierras,"POLYGON ((-118.71895 37.07419, -118.71895 37.0..."
1,000617d8-8c14-43e2-b708-7e3a69fe3cc3,central rockies,"POLYGON ((-107.07679 37.78042, -107.07679 37.7..."
2,000863e7-21e6-477d-b799-f5675c348627,other,"POLYGON ((-119.40167 37.02400, -119.40167 37.0..."
3,000ba8d9-d6d5-48da-84a2-1fa54951fae1,sierras,"POLYGON ((-119.32082 37.43171, -119.32082 37.4..."
4,00146204-d4e9-4cd8-8f86-d1ef133c5b6d,sierras,"POLYGON ((-118.52132 36.65735, -118.52132 36.6..."
...,...,...,...
18125,ffdfb5a4-91a0-41a9-a4d5-501b04ef6326,sierras,"POLYGON ((-118.62014 37.11718, -118.62014 37.1..."
18126,ffe43514-2c92-43b6-bd84-d183806aca65,other,"POLYGON ((-123.49799 47.90132, -123.49799 47.9..."
18127,ffeabc13-7c6f-4b63-b043-19c8f15e0345,sierras,"POLYGON ((-119.64422 37.87976, -119.64422 37.8..."
18128,fff95195-ccc9-40b7-b302-a0d8570c86bc,other,"POLYGON ((-123.37223 47.73242, -123.37223 47.7..."


In [33]:
centroids = np.vstack(df_grid.geometry.centroid).T
lon_grid = centroids[0:1].ravel().tolist()
lat_grid = centroids[1:2].ravel().tolist()


In [34]:
grid_labels = df_grid['cell_id'].to_numpy()
grid_labels = grid_labels.ravel().tolist()

In [39]:
dataUA_grid  = ds_ua.sel(lon = lon_grid, lat = lat_grid, method='nearest')
dataUA_grid = dataUA_grid.assign_coords(label = grid_labels) 

In [40]:
dataUA_grid

<xarray.Dataset>
Dimensions:   (label: 18130, lat: 18130, lon: 18130, time: 1826, time_str_len: 11)
Coordinates:
  * lat       (lat) float32 37.083332 37.791668 37.041668 37.416668 ...
  * lon       (lon) float32 -118.708336 -107.083336 -119.416664 -119.333336 ...
  * time      (time) datetime64[ns] 2012-10-01 2012-10-02 2012-10-03 ...
  * label     (label) <U36 '0003f387-71c4-48f6-b2b0-d853bd4f0aba' ...
Dimensions without coordinates: time_str_len
Data variables:
    crs       (time) |S1 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' ...
    time_str  (time_str_len, time) |S1 dask.array<shape=(11, 1826), chunksize=(11, 365)>
    SWE       (time, lat, lon) float32 dask.array<shape=(1826, 18130, 18130), chunksize=(365, 18130, 18130)>
    DEPTH     (time, lat, lon) float32 dask.array<shape=(1826, 18130, 18130), chunksize=(365, 18130, 18130)>

In [42]:
#SWE_UA = dataUA_grid.SWE.sel(time = '2016-02-02')
#im = SWE_UA.plot()

In [46]:
ds_ua.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

ValueError: Coordinates {coords} must all have the same dimensions in the same order

In [47]:
ds_ua

<xarray.Dataset>
Dimensions:   (lat: 621, lon: 1405, time: 1826, time_str_len: 11)
Coordinates:
  * lat       (lat) float32 24.083334 24.125 24.166666 24.208334 24.25 ...
  * lon       (lon) float32 -125.0 -124.958336 -124.916664 -124.875 ...
  * time      (time) datetime64[ns] 2012-10-01 2012-10-02 2012-10-03 ...
Dimensions without coordinates: time_str_len
Data variables:
    crs       (time) |S1 b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' ...
    time_str  (time_str_len, time) |S1 dask.array<shape=(11, 1826), chunksize=(11, 365)>
    SWE       (time, lat, lon) float32 dask.array<shape=(1826, 621, 1405), chunksize=(365, 621, 1405)>
    DEPTH     (time, lat, lon) float32 dask.array<shape=(1826, 621, 1405), chunksize=(365, 621, 1405)>

In [48]:
ds_mesh

<xarray.Dataset>
Dimensions:  (x: 100, y: 100)
Coordinates:
    lat      (x, y) float64 19.31 27.73 -10.93 1.032 -18.08 -76.77 86.78 ...
    lon      (x, y) float64 172.2 -91.2 56.69 -127.9 -45.15 -44.48 -5.636 ...
Dimensions without coordinates: x, y
Data variables:
    field    (x, y) float64 191.5 -63.46 45.76 -126.8 -63.22 -121.2 81.14 ...